# Recognicition of The Office Quotes

For the NLP project we decided to get a Dataset consisting of 2 csv files:
- parent.csv :

    These are all quotes when the character was replying to someone else from the show.

- talking_head.csv :

    These are all quotes, from when the characters were directly interacting with the camera.

All the Quotes are from 4 main characters of the show, meaning they are the ones with the more prominent speaking roles:
- Michael
- Pam
- Jim
- Dwight

## Description of Dataset

Before we start with the Preprocessing steps and working with the Data in an NLP sense. We will take a look into the Data and see what exactly it offers and how balanced it is.